<a href="https://colab.research.google.com/github/danielmachlab/imdb_review_classifier/blob/development/imdb_review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMDB Review Classifier

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras

TensorFlow 1.x selected.
